# Generating synthetic data sets of a river bed column forced by monoperiodic river and aquifer temperature signals

2024 developped by the student group from ``demo_genData.ipynb`` created by Nicolas Flipo during MOLONARI 2024 and ``demo_Pyheatmy.ipynb``.

The object of this demo is to test the validity of our MCMC inversion.

In order to do so, we first generate a virtual point and its data thanks to ``demo_genData.ipynb`` and then inverse those data to find the parameters of the point. We can then see if we find back the parameters we used to generate the virtual point. 

In [ ]:
from pyheatmy import *
import pandas as pd

NBITER = 10

## Génération et traitement d'un jeu de données virtuelles

On exécute le notebook ``demo_genData.ipynb`` pour générer un jeu de données dont on connaîtra les caractéristiques.

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from IPython.display import display, clear_output

path_demoGenData = "demo_genData.ipynb"

# Fonction pour exécuter un notebook
def run_notebook(notebook_path):
    # Charger le notebook
    with open(notebook_path) as f:
        notebook = nbformat.read(f, as_version=4)

    # Configurer l'ExecutePreprocessor pour exécuter toutes les cellules
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')

    # Exécuter le notebook
    try:
        ep.preprocess(notebook, {'metadata': {'path': './'}})
        print("Notebook exécuté avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'exécution du notebook : {e}")

# Fonction pour importer les valeurs des cellules de code d'un notebook dans le contexte global
def import_notebook_values(notebook_path):
    # Charger le notebook
    with open(notebook_path) as f:
        notebook = nbformat.read(f, as_version=4)

    # Créer un dictionnaire pour contenir les variables globales du notebook
    notebook_globals = {}

    try:
        # Exécuter chaque cellule de code
        for cell in notebook.cells:
            if cell.cell_type == 'code':
                exec(cell.source, notebook_globals)
                
        # Maintenant, importer les variables du notebook dans l'espace global
        globals().update(notebook_globals)  # Met à jour l'espace global avec les variables du notebook
        print("Les valeurs du notebook sont maintenant accessibles.")
        
    except Exception as e:
        print(f"Erreur lors de l'exécution du notebook : {e}")

run_notebook(path_demoGenData)
print("")
print("Affichage temporaire")
print("")
import_notebook_values(path_demoGenData)
clear_output()

On extrait ensuite et traite les données générées par le notebook, de la même façon qu'elles sont traitées dans ``demo_Pyheatmy.ipynb``.

In [ ]:
# define the folder path
folder = col._dir_print + '/VirtualPoint'

# read the data
capteur_riviere = pd.read_csv(folder + "/VirtualPoint_P_measures.csv", sep = ',', names = ['dates', 'tension', 'temperature_riviere'], skiprows=1)
capteur_ZH = pd.read_csv(folder + "/VirtualPoint_T_measures.csv", sep = ',', names = ['dates', 'temperature_10', 'temperature_20', 'temperature_30', 'temperature_40'], skiprows=1)
etalonage_capteur_riv = pd.read_csv("./configuration/pressure_sensors/P508.csv")

On définit une fonction pour convertir les dates des dataframe, cela peut éviter certains problèmes.

In [ ]:
def convertDates(df: pd.DataFrame):
    """
    Convert dates from a list of strings by testing several different input formats
    Try all date formats already encountered in data points
    If none of them is OK, try the generic way (None)
    If the generic way doesn't work, this method fails
    (in that case, you should add the new format to the list)
    
    This function works directly on the giving Pandas dataframe (in place)
    This function assumes that the first column of the given Pandas dataframe
    contains the dates as characters string type
    
    For datetime conversion performance, see:
    See https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor
    """
    formats = ("%m/%d/%y %H:%M:%S", "%m/%d/%y %I:%M:%S %p",
               "%d/%m/%y %H:%M",    "%d/%m/%y %I:%M %p",
               "%m/%d/%Y %H:%M:%S", "%m/%d/%Y %I:%M:%S %p", 
               "%d/%m/%Y %H:%M",    "%d/%m/%Y %I:%M %p",
               "%y/%m/%d %H:%M:%S", "%y/%m/%d %I:%M:%S %p", 
               "%y/%m/%d %H:%M",    "%y/%m/%d %I:%M %p",
               "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %I:%M:%S %p", 
               "%Y/%m/%d %H:%M",    "%Y/%m/%d %I:%M %p",
               None)
    times = df[df.columns[0]]
    for f in formats:
        try:
            # Convert strings to datetime objects
            new_times = pd.to_datetime(times, format=f)
            # Convert datetime series to numpy array of integers (timestamps)
            new_ts = new_times.values.astype(np.int64)
            # If times are not ordered, this is not the appropriate format
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0 :
                #print("Order is not the same")
                raise ValueError()
            # Else, the conversion is a success
            #print("Found format ", f)
            df[df.columns[0]] = new_times
            return
        
        except ValueError:
            #print("Format ", f, " not valid")
            continue
    
    # None of the known format are valid
    raise ValueError("Cannot convert dates: No known formats match your data!")

convertDates(capteur_riviere)
convertDates(capteur_ZH)

On définit les objets ``dH_measures`` et ``T_measures``, nécessaires à la création d'un objet ``Column``, qui réalisera les calculs :
- ``dH_measures`` contient les dates des mesures, les mesures de différence de charge, et les températures de la rivière.
- ``T_measures`` contient les dates des mesures, et les températures mesurées par les 4 capteurs de la tige.

In [ ]:
# set seed for reproducibility
np.random.seed(0)

# conversion des mesures de pression
intercept = float(etalonage_capteur_riv['P508'][2])
a = float(etalonage_capteur_riv['P508'][3])
b = float(etalonage_capteur_riv['P508'][4])
capteur_riviere['dH'] = (capteur_riviere['tension'].astype(float)-intercept-capteur_riviere['temperature_riviere'].astype(float)*b)/a

# conversion mesures de tempétratures
capteur_riviere['temperature_riviere'] = capteur_riviere['temperature_riviere'] + ZERO_CELSIUS
capteur_ZH['temperature_10'] = capteur_ZH['temperature_10'] + ZERO_CELSIUS
capteur_ZH['temperature_20'] = capteur_ZH['temperature_20'] + ZERO_CELSIUS
capteur_ZH['temperature_30'] = capteur_ZH['temperature_30'] + ZERO_CELSIUS
capteur_ZH['temperature_40'] = capteur_ZH['temperature_40'] + ZERO_CELSIUS

# définition des attributs de colonnes
dH_measures = list(zip(capteur_riviere['dates'],list(zip(capteur_riviere['dH'], capteur_riviere['temperature_riviere']))))
T_measures = list(zip(capteur_ZH['dates'], capteur_ZH[['temperature_10', 'temperature_20', 'temperature_30', 'temperature_40']].to_numpy()))

print(f"dH : {dH_measures}")
print(f"Tshaft : {T_measures}")


On définit maintenant l'objet ``Column`` à partir d'un dictionnaire.

In [ ]:
Zbottom = 0.4

col_dict = {
	"river_bed": 1., 
    "depth_sensors": [.1, .2, .3, Zbottom],
	"offset": .0,
    "dH_measures": dH_measures,
	"T_measures": T_measures,
    "sigma_meas_P": None,
    "sigma_meas_T": None,
    "inter_mode": 'lagrange'
}

col = Column.from_dict(col_dict)

## Inversion des données générées

On va maintenant inverser les données générées pour obtenir les différentes caractéristiques.

In [ ]:
NBITERMCMC = 20

priors = {
    "Prior_IntrinK": ((1e-14, 1e-8), 5e-10), # (intervalle, sigma)
    "Prior_n": ((.01, .25), .0125),
    "Prior_lambda_s": ((1, 10), .5),
    "Prior_rhos_cs": ((1e6,1e7), 5e5),
    "Prior_q_s": ((-1e-6,1e-6), 1e-12),
}

Layer_homogenous = {
    "name": "Couche en sable",
    "zLow": Zbottom,
    "IntrinK":1e-11,
    "n": 0.1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q_s": -1e-8,
}

Layer_homogenous = Layer.from_dict(Layer_homogenous)

# On crée une couche de référence pour la comparaison plus bas
reference_params_dict = {
    "name": "Couche en sable",
    "zLow": Zbottom,
    "IntrinK":1e-11,
    "n": 0.1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q_s": -1e-8,
}

layer_for_simulation = Layer.from_dict(reference_params_dict)

col.set_layers(Layer_homogenous)
Layer_homogenous.set_priors_from_dict(priors)
print(Layer_homogenous.Prior_list)


col.all_layers

col.compute_mcmc(
    nb_iter = NBITERMCMC,
    verbose=True
)

bestLayers = col.get_best_layers()

col.plot_all_param_pdf()


## Comparaison des paramètres réels et de ceux inversés

On peut à présent comparer les données inversées et celles orignales.

In [ ]:
import pandas as pd

# --- ÉTAPE 1 : ACTION (inchangée) ---
col.get_best_layers()

# --- ÉTAPE 2 : RÉCUPÉRATION (inchangée) ---
inverted_params = col.get_list_current_params()[0]

# --- ÉTAPE 3 : COMPARAISON (corrigée) ---
# On utilise maintenant le dictionnaire de référence pour les "vraies" valeurs.
data = {
    'Paramètre': PARAM_LIST,
    'Valeur de Référence': [
        reference_params_dict['IntrinK'],
        reference_params_dict['n'],
        reference_params_dict['lambda_s'],
        reference_params_dict['rhos_cs'],
        reference_params_dict['q_s']
    ],
    'Valeur Inversée (MCMC)': [
        inverted_params.IntrinK,
        inverted_params.n,
        inverted_params.lambda_s,
        inverted_params.rhos_cs,
        inverted_params.q_s
    ]
}

# Créer et afficher le tableau de comparaison
comparison_df = pd.DataFrame(data)

# Appliquer un formatage pour une meilleure lisibilité
comparison_df['Valeur de Référence'] = comparison_df['Valeur de Référence'].map('{:.3e}'.format)
comparison_df['Valeur Inversée (MCMC)'] = comparison_df['Valeur Inversée (MCMC)'].map('{:.3e}'.format)

print("--- Comparaison des paramètres de référence et inversés ---")
print(comparison_df.to_string(index=False))

## Inversion des données dans le cas d'un paramètre fixé (ici $q_s$)

In [ ]:
NBITERMCMC = 20

priors = {
    "Prior_IntrinK": ((1e-14, 1e-8), 1e-11), # (intervalle, sigma)
    "Prior_n": ((.01, .25), .0125),
    "Prior_lambda_s": ((1, 10), 0.5),
    "Prior_rhos_cs": ((1e6,1e7), 5e5),
    "Prior_q_s": (-1e-8, 0),
}

Layer_homogenous = {
    "name": "Couche en sable",
    "zLow": Zbottom,
    "IntrinK":1e-11,
    "n": 0.1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q_s": -1e-8,
}

Layer_homogenous = Layer.from_dict(Layer_homogenous)

# On crée une couche de référence pour la comparaison plus bas
reference_params_dict = {
    "name": "Couche en sable",
    "zLow": Zbottom,
    "IntrinK":1e-11,
    "n": 0.1,
    "lambda_s": 2,
    "rhos_cs": 4e6,
    "q_s": -1e-8,
}

layer_for_simulation = Layer.from_dict(reference_params_dict)

col.set_layers(Layer_homogenous)
Layer_homogenous.set_priors_from_dict(priors)
print(Layer_homogenous.Prior_list)


col.all_layers

col.compute_mcmc(
    nb_iter = NBITERMCMC,
    verbose=True
)

bestLayers = col.get_best_layers()

col.plot_all_param_pdf()


In [ ]:
import pandas as pd

# --- ÉTAPE 1 : ACTION (inchangée) ---
col.get_best_layers()

# --- ÉTAPE 2 : RÉCUPÉRATION (inchangée) ---
inverted_params = col.get_list_current_params()[0]

# --- ÉTAPE 3 : COMPARAISON (corrigée) ---
# On utilise maintenant le dictionnaire de référence pour les "vraies" valeurs.
data = {
    'Paramètre': PARAM_LIST,
    'Valeur de Référence': [
        reference_params_dict['IntrinK'],
        reference_params_dict['n'],
        reference_params_dict['lambda_s'],
        reference_params_dict['rhos_cs'],
        reference_params_dict['q_s']
    ],
    'Valeur Inversée (MCMC)': [
        inverted_params.IntrinK,
        inverted_params.n,
        inverted_params.lambda_s,
        inverted_params.rhos_cs,
        inverted_params.q_s
    ]
}

# Créer et afficher le tableau de comparaison
comparison_df = pd.DataFrame(data)

# Appliquer un formatage pour une meilleure lisibilité
comparison_df['Valeur de Référence'] = comparison_df['Valeur de Référence'].map('{:.3e}'.format)
comparison_df['Valeur Inversée (MCMC)'] = comparison_df['Valeur Inversée (MCMC)'].map('{:.3e}'.format)

print("--- Comparaison des paramètres de référence et inversés ---")
print(comparison_df.to_string(index=False))